In [1]:
open class Day(private val dayNum: Int) {
    val dataFolder = java.io.File("data21")
    fun readDay(number: Int) = dataFolder.resolve("day$number.txt").readText().lines()
    protected val input by lazy { readDay(dayNum) }
    
    open fun part1(): Any {
        return 0
    }
    
    open fun part2(): Any {
        return 0
    }
    
    fun render(): Any {
        return HTML("""
            <h3 id="day$dayNum">Day $dayNum.</h2>
            <p><b>Part 1 answer:</b> ${part1()}</p>
            <p><b>Part 2 answer:</b> ${part2()}</p>
        """.trimIndent())
    }
}

fun day(number: Int, p1: (List<String>) -> Int, p2: (List<String>) -> Int): Day {
    return object : Day(number) {
        override fun part1(): Int {
            return p1(input)
        }
        override fun part2(): Int {
            return p2(input)
        }
    }
}

USE {
    render<Day> { it.render() }
}

In [2]:
object : Day(0) {
    override fun part1(): Int {
        return 0
    }
    
    override fun part2(): Int {
        return 0
    }
}

Day 0. 
 Part 1 answer: 0 
 Part 2 answer: 0

In [3]:
object : Day(1) {
    val inputList = input.map { it.toInt() }
    fun List<Int>.howMany() = subList(1, size).zip(subList(0, size - 1)).count {(after, before) -> after > before}
    
    override fun part1(): Int {
        return inputList.howMany()
    }
    
    override fun part2(): Int {
        return inputList.windowed(3).map { it.sum() }.howMany()
    }
}

Day 1. 
 Part 1 answer: 1616 
 Part 2 answer: 1645

In [4]:
object : Day(2) {
    override fun part1(): Int {
        var x = 0
        var d = 0

        input.forEach {
            val command = it.substringBefore(' ')
            val value = it.substringAfter(' ').toInt()
            when(command) {
                "up" -> d -= value
                "down" -> d += value
                "forward" -> x += value
            }
        }
        return x * d
    }
    
    override fun part2(): Int {
        var x = 0
        var d = 0
        var aim  = 0
        input.forEach {
            val command = it.substringBefore(' ')
            val value = it.substringAfter(' ').toInt()
            when(command) {
                "up" -> aim -= value
                "down" -> aim += value
                "forward" -> {
                    x += value
                    d += aim * value
                }
            }
        }
        return x * d
    }
}

Day 2. 
 Part 1 answer: 1728414 
 Part 2 answer: 1765720035

In [5]:
object : Day(3) {
    val n = input.first().length
    val m = input.size
    
    fun List<Int>.convert() = fold(0) {acc, b -> acc * 2 + b}
    
    fun List<String>.myFilter(bitI: Int, mostCommon: Boolean): List<String> {
        val n = size
        val r = map { it[bitI] == '1' }
        val onesCount = r.count { it }
        val isOnes = onesCount * 2 >= n
        return filterIndexed { i, _ -> r[i] xor isOnes xor mostCommon  }
    }

    fun List<String>.myFold(mostCommon: Boolean) = foldIndexed(this) { bitI, acc, _ ->
        if (acc.size <= 1) acc else { acc.myFilter(bitI, mostCommon) }
    }.single().map {if(it == '0') 0 else 1}.convert()
    
    override fun part1(): Int {
        val result = input.fold(MutableList(n){0}) { acc, line ->
            line.forEachIndexed { i, c -> if (c == '1') acc[i]++ };
            acc 
        }.map { if (it * 2 >= m) 1 else 0 }
        val iresult = result.map { 1 - it }
        return result.convert() * iresult.convert()
    }
    
    override fun part2(): Int {
        return input.myFold(true) * input.myFold(false)
    }
}

Day 3. 
 Part 1 answer: 2035764 
 Part 2 answer: 2817661

In [6]:
class Board(val data: List<List<Int>>) {
    val mask = MutableList(5) { MutableList(5) { false } }
    
    fun setNum(n: Int) {
        data.forEachIndexed { i, row ->
            row.forEachIndexed { j, num -> 
                if (n == num) mask[i][j] = true
            }
        }
    }
    
    fun check(): Boolean {
        if (mask.any { row -> row.all { it } }) return true
        
        for (i in 0..4) {
            if(mask.all { it[i] }) return true
        }
        
        return false
    }
    
    fun sumUnmarked(): Int {
        return data.mapIndexed { rowI, row -> row.filterIndexed { i, _ -> !mask[rowI][i] } }.flatten().sum()
    }
}

object : Day(4) {
    val requests = input.first().split(",").map {it.toInt()}
    val splitRegex = " +".toRegex()
    val boards = (2 .. (input.size - 2) step 6).map { start ->
        val borderLines = input.subList(start, start + 5)
        // println(borderLines)
        Board(borderLines.map { it.split(splitRegex).filter {!it.isBlank()} .map { it.toInt() } })
    }
    
    override fun part1(): Int {        
        for (r in requests) {
            for (b in boards) {
                b.setNum(r)
                if (b.check()) {
                    return b.sumUnmarked() * r                    
                }
            }
        }
        return 0
    }
    
    override fun part2(): Int {
        val winners = mutableSetOf<Int>()
        var lastScore  = -1
        
        for (r in requests) {
            for ((ib, b) in boards.withIndex()) {
                if (ib in winners) continue
                b.setNum(r)
                if (b.check()) {
                    winners.add(ib)
                    lastScore = b.sumUnmarked() * r                    
                }
            }
        }
        return lastScore
    }
}

Day 4. 
 Part 1 answer: 74320 
 Part 2 answer: 17884

In [7]:
object : Day(5) {
    val n = 1000
    
    inner class Line(
        val x1: Int,
        val y1: Int,
        val x2: Int,
        val y2: Int,
    ) {
        fun points(): List<Pair<Int, Int>> {
            return buildList {
                val mi = (x2 - x1).sign
                val mj = (y2 - y1).sign
                var x = x1
                var y = y1
                val n = kotlin.math.max((x2 - x1).absoluteValue, (y2 - y1).absoluteValue)

                for(i in 0..n) {
                    add(x to y)
                    x += mi
                    y += mj
                }
            }
        }

        val isDiagonal get() = x1 != x2 && y1 != y2
    }
    
    val lines = input.map { line -> 
        val (p1, p2) = line.split(" -> ")
        val (x1, y1) = p1.split(",").map {it.toInt()}
        val (x2, y2) = p2.split(",").map {it.toInt()}
        Line(x1, y1, x2, y2)
    }
    
    fun MutableList<IntArray>.mark(pt: Pair<Int, Int>) {
        val v = this[pt.first][pt.second]
        this[pt.first][pt.second] = if (v == 0) 1 else 2
    }
    
    override fun part1(): Int {
        val a = MutableList(n) { IntArray(n) }
        for(line in lines) {
            if (line.isDiagonal) continue
            line.points().forEach { pt ->
                a.mark(pt)
            }
        }
        return a.sumOf { it.count { it == 2 }}
    }
    
    override fun part2(): Int {
        val a = MutableList(n) { IntArray(n) }
        for(line in lines) {
            line.points().forEach { pt ->
                a.mark(pt)
            }
        }
        return a.sumOf { it.count { it == 2 }}
    }
}

Day 5. 
 Part 1 answer: 4421 
 Part 2 answer: 18674

In [8]:
object : Day(6) {    
    val fishes = input.first().split(",").map { it.toInt() }
    
    override fun part1(): Int {
        var state = fishes
        var born = 0;
        for (i in 1..80) {
            val newFishes = state.map { if (it == 0) 6 else it - 1 }
            state = newFishes + MutableList(born) { 8 }
            born = newFishes.count {it == 0}
        }
        return state.size
    }
    
    override fun part2(): Long {
        var state = MutableList(9) { 0L }
        fishes.forEach { state[it]++ }
        for (i in 1..256) {
            state = MutableList(9) { if (it == 8) state[0] else if(it == 6) state[7] + state[0] else state[it + 1] }
        }
        return state.sum()
    }
}

Day 6. 
 Part 1 answer: 389726 
 Part 2 answer: 1743335992042

In [9]:
object : Day(7) {
    val a = input.first().split(",").map { it.toInt() }
    
    override fun part1(): Any {
        return a.minOf { el -> a.sumOf { kotlin.math.abs(it - el) } }
    }
    
    override fun part2(): Any {
        return a.minOf { el -> a.sumOf { val k = kotlin.math.abs(it - el); k.toLong() * (k + 1)/2 } }
    }
}

Day 7. 
 Part 1 answer: 355764 
 Part 2 answer: 99634572